# Chapter 03: Pure Happiness with Pure Functions

### Pure Function definition:
A pure function is a function that consistently produces the same output for the same input and has no observable side effects.

The key properties of pure functions are:

1. **Cacheable:** \
Results can be cached since the output depends solely on the input.

2. **Portable / Self-documenting:** \
The behavior is predictable and easy to understand without external context.

3. **Testable:** \
Testing is straightforward due to deterministic outputs.

4. **Reasonable:** \
The logic is isolated, making it easy to reason about and debug.

5. **Parallel:** \
Pure functions can be executed in parallel without risk of interference.
---

#### Exercising with examples of pure & unpure functions

In [23]:
list = [1, 2, 3]
print(f"List: {list}\n")

def notPureFct():
    return list.pop()

print(f"Output of a not pure function 1st execution: {notPureFct()}")
print(f"Output of a not pure function 2nd execution: {notPureFct()}")
print(f"Let's view the list once again : {list}")

List: [1, 2, 3]

Output of a not pure function 1st execution: 3
Output of a not pure function 2nd execution: 2
Let's view the list once again : [1]


In [24]:
list = [1, 2, 3]
print(f"List: {list}\n")

def pureFct(l):
    return l[:].pop()

print(f"Output of a pure function 1st execution: {pureFct(list)}")
print(f"Output of a pure function 2nd execution: {pureFct(list)}")
print(f"Let's view the list once again : {list}")

List: [1, 2, 3]

Output of a pure function 1st execution: 3
Output of a pure function 2nd execution: 3
Let's view the list once again : [1, 2, 3]


---
#### Cacheability of pure functions

In [4]:
import time
import json
from functools import cache

In [8]:
# Pure functions are cacheable
# del cached_factorial

@cache
def cached_factorial(i=0):
    if i==0:
        return 1
    time.sleep(1)
    return cached_factorial(i-1)*i

In [9]:
start_time = time.time()
output = cached_factorial(5)
print(f"First execution of cached function took {(time.time()-start_time):.2f} second\n{output}")

start_time = time.time()
output = cached_factorial(5)
print(f"Second execution of cached function took {(time.time()-start_time):.2f} second\n{output}")

First execution of cached function took 5.00 second
120
Second execution of cached function took 0.00 second
120


In [12]:
# For a manual implementation of cache
def memoize(function):
    cache = dict()
    
    def wrapper(*args):
        str_args = json.dumps(args)
        cache[str_args] = cache[str_args] if str_args in cache else function(*args)
        return cache[str_args]

    return wrapper

In [14]:
@memoize
def my_factorial(i=0):
    if i==0:
        return 1
    time.sleep(1)
    return my_factorial(i-1)*i

In [ ]:
start_time = time.time()
f = my_factorial(5)
print(f"First execution of my function took {(time.time()-start_time):.2f} second\n{f}")

start_time = time.time()
f = my_factorial(7)
print(f"Second execution of my function took {(time.time()-start_time):.2f} second\n{f}")

Start calculation
First execution of my function took 5.00 second
120
Second execution of my function took 2.00 second
5040


The function calculated <u>my_factorial(7)</u> more quickly because it reused the results of previous executions up to <u>my_factorial(5)</u>.

---
#### Portability of pure functions

Let's start by creating an external dependency

In [20]:
dependency = 5

def nonPure(x):
    return dependency+x

def pure(dependency, x):
    return dependency + x

In [21]:
print(nonPure(2))
dependency = 3
print(nonPure(2))

7
5


This following code might appear straightforward and repetitive, but it incorporates the principle of portability.

In [22]:
# This code might seem straightforward and repetitive but it incorporate the concept of portability
print(pure(5, 2))
dependency = 122
print(pure(5, 2))

7
7


---
#### Reasonable

In [39]:
jobe = { 'name': 'Jobe', 'hp': 20, 'team': 'red' }
michael = { 'name': 'Michael', 'hp': 20, 'team': 'green' }

decrementHP = lambda p: p.update({'hp': p.pop('hp')-1})
isSameTeam = lambda p1, p2: p1['team'] == p2['team']
punch = lambda a, t:  t if isSameTeam(a, t) else decrementHP(t)

punch(jobe, michael) 
print(michael)

{'name': 'Michael', 'team': 'green', 'hp': 19}


In [40]:
# inline the function isSameTeam
punch = lambda a, t: t if a['team'] == t['team'] else decrementHP(t)
punch(jobe, michael) 
print("jobe 2nd punch on michael: ", michael)

# It gets converted to the following check
punch = lambda a, t: t if 'red' == 'green' else decrementHP(t)
punch(jobe, michael) 
print("jobe 3rd punch on michael: ", michael)

# Since it's false we can remove the entire if branch
punch = lambda a, t: decrementHP(t)
punch(jobe, michael) 
print("jobe isn't going easy on michael: ", michael)

# if we inline decrementHP
punch = lambda a, t: t.update({'hp': t.pop('hp')-1})
punch(jobe, michael) 
print("jobe is ruthless: ", michael)

jobe 2nd punch on michael:  {'name': 'Michael', 'team': 'green', 'hp': 18}
jobe 3rd punch on michael:  {'name': 'Michael', 'team': 'green', 'hp': 17}
jobe isn't going easy on michael:  {'name': 'Michael', 'team': 'green', 'hp': 16}
jobe is ruthless:  {'name': 'Michael', 'team': 'green', 'hp': 15}
